# 02 - stdio Deep Dive (script-first)

This notebook validates stdio subprocess connectivity and common MCP calls against a server.

Targets:
- connect (initialize)
- tools/resources/prompts
- tool calls + rendering
- inspect logs + stderr tail


In [ ]:
# Ensure local workspace import
import sys
from pathlib import Path

def find_root(start: Path) -> Path:
    for cur in [start, *start.parents]:
        if (cur / "pyproject.toml").exists():
            return cur
    return start

ROOT = find_root(Path.cwd().resolve())
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

import mcp_tuning
print("python:", sys.executable)
print("cwd:", Path.cwd())
print("mcp_tuning:", mcp_tuning.__file__)


## Connect

Pick one server command. Recommended for validation:
- mock server (no external deps): `python src/mcp_tuning/_mock_server.py`
- FastMCP example (requires `mcp` installed in the interpreter running the server): `python examples/servers/simple_stdio_server.py`


In [ ]:
from mcp_tuning import connect_stdio
import sys

# Default: mock server
c = connect_stdio([sys.executable, "src/mcp_tuning/_mock_server.py"])
c.server_info()


In [ ]:
# List tools
tools = c.tools()
[t.name for t in tools]


In [ ]:
# Call a tool
r = c.call("add", {"a": 1, "b": 2})
r.ok, r.result, r.error


In [ ]:
# Render result
c.show(r.result)


In [ ]:
# Inspect logs
events = c.inspector.events_tail(20)
[(e.ts, e.direction, e.message.get("method"), e.meta.get("duration_ms")) for e in events]


In [ ]:
# Clean up
c.close()
